# AB SEO Test Summary

In [2]:
!pip install pyathena

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from pyathena import connect

In [10]:
from smart_open import smart_open

ModuleNotFoundError: No module named 'smart_open'

In [2]:


import pandas as pd
import numpy as np
from datetime  import datetime, timedelta, date
import time


from athena_querying import *
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

## Settings

In [3]:
run_speed_tests = True

# Get Results

In [4]:

athena_conn, athena_cursor = connect_to_athena()

In [5]:
print("Checking athena works")
query = "select * from ms_data_stream_production_processed limit 100"
test_query_results = pd.read_sql(query, athena_conn)

test_query_results.head(10)

Checking athena works


,type,context,user,body,sent_at
0,event,"{title=gak daftarin bpjs ketenagakerjaan, ini ...",{anonymous_id=3421817d-d41f-4814-b8e8-2dbb60c6...,"{event_name=Reading, data={title=gak daftarin ...",2019-06-21T13:02:52.025Z
1,event,{title=I Dewa Gede Palguna dan Dua Mobilnya Se...,{anonymous_id=659317f7-2a9a-4828-b672-9e2e84f4...,"{event_name=Reading, data={title=I Dewa Gede P...",2019-06-21T13:02:53.336Z
2,event,{title=Suti Karno alias Atun Geluti 3 Bisnis y...,{anonymous_id=f1fcee74-aa90-47c9-b1a6-a47d78db...,"{event_name=Reading, data={title=Suti Karno al...",2019-06-21T13:02:55.108Z
3,event,"{title=Deddy Corbuzier Mualaf, Ini 4 Fakta Men...",{anonymous_id=945625de-95c7-448b-afc7-e068dbda...,"{event_name=Reading, data={title=Deddy Corbuzi...",2019-06-21T13:02:53.681Z
4,event,{title=Cara Ambil Antrian Online BPJS Ketenaga...,{anonymous_id=92b069d4-845d-4fca-ae85-07cd0e8b...,"{event_name=Reading, data={title=Cara Ambil An...",2019-06-21T13:02:53.122Z
5,event,"{title=6 Saham Perusahaan Pengusaha Surabaya, ...",{anonymous_id=35374d92-db83-47e3-b6a3-5cbc2b7d...,"{event_name=Reading, data={title=6 Saham Perus...",2019-06-21T13:02:24.658Z
6,event,{title=mau balik nama motor? ini biaya yang ha...,{anonymous_id=c64d2994-95c2-4657-af98-69a04056...,"{event_name=Reading, data={title=mau balik nam...",2019-06-21T13:02:52.932Z
7,event,{title=Irish Bella dan Potret Rumah Mewahnya y...,{anonymous_id=b80818f7-b37f-499e-b7bd-450b470c...,"{event_name=Reading, data={title=Irish Bella d...",2019-06-21T13:02:54.087Z
8,event,{title=Perang Fashion Kahiyang Ayu Vs Aliya Ra...,{anonymous_id=0f773083-bcfa-4a14-84a7-8a7dee9a...,"{event_name=Reading, data={title=Perang Fashio...",2019-06-21T13:02:54.153Z
9,event,{title=semboyan orang makassar yang jadi motiv...,{anonymous_id=d7a4c66f-e648-41ee-a9fe-cbb40796...,"{event_name=Reading, data={title=semboyan oran...",2019-06-21T13:02:58.677Z


In [9]:
# blog_pageview_metrics
# blog_pageviews_per_user_per_day

# Speed Testing

In [6]:
speed_test_query = "select * from "+athena_database +".blog_pageview_metrics order by day_str, is_test"

In [ ]:
# Original / obvious way (may be dependent on imports that have been changed in other files)
if run_speed_tests:
    start_time = datetime.now()
    
    pageview_metrics_results = pd.read_sql(speed_test_query, athena_conn)
    end_time = datetime.now()
    execution_seconds_conventional = (end_time-start_time).total_seconds()
    print("Running the old way took %.2f seconds"%execution_seconds_conventional)
    pageview_metrics_results.head()


In [ ]:
# TODO: sounds like you can speed up athena massively by querying from the S3 bucket, 
# rather than getting results from the connection object.  There's some issue with long running queries and large results.
# https://github.com/laughingman7743/PyAthena/issues/46
# https://stackoverflow.com/questions/53262533/why-pyathena-doesnt-work-on-longer-running-queries-while-athena-runs-them/53324034

In [9]:
# Using pandas cursor 
if run_speed_tests:
    ap_start_time = datetime.now()
    athena_cursor.execute(speed_test_query)
    ap_query_done_time = datetime.now()
    query_duration_seconds = (ap_query_done_time - ap_start_time).total_seconds()
    print("Query finished after %.2f" % query_duration_seconds)
    
    df = as_pandas(athena_cursor)
    ap_done_time = datetime.now()
    ap_total_seconds = (ap_done_time - ap_start_time).total_seconds()
    print("All finished using as_pandas in %.2f"%ap_total_seconds)
    df.head()

Query finished after 30.87
All finished using as_pandas in 582.44


In [20]:
# Querying the S3 results directly
if run_speed_tests:
    # assume query has been run and timed already
    # Note that this way it doesn't have context on the data types, but I'm not sure glue was set up properly for that anyway.
    s3_start_time = datetime.now()
    bucket, key = athena_cursor.output_location.strip('s3://').split('/', 1)
    print(bucket,key)
    session = boto3.session.Session()

    # result = session.resource('s3').Object(bucket, key)
    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=bucket,Key=key)
    
    file = response["Body"]

    df = pd.read_csv(file)
    s3_end_time = datetime.now()
    s3_read_seconds = (s3_end_time - s3_start_time).total_seconds()
    print("Reading the CSV from S3 (not running query) took %.2f"%s3_read_seconds)

aws-athena-query-results-358002497134-ap-southeast-1 9a384bb0-0231-4738-993a-1ad97dcc45f0.csv
Reading the CSV from S3 (not running query) took 10.80


In [17]:
df

,anonymous_id,canonical_url,day_str,from_time,to_time,time_on_page_seconds,is_test,max_article_read_depth
0,752366de-ce71-4e93-9fda-a481288a5ef9,https://www.moneysmart.id/jusuf-kalla-dengan-t...,2010-01-01,2010-01-01 00:01:56.434,2010-01-01 00:01:56.434,0,False,0
1,42cee154-32e1-4a88-aa1b-c0a8c9b8bd55,https://www.moneysmart.id/daftar-mobil-termaha...,2016-01-01,2016-01-01 00:04:09.670,2016-01-01 00:04:09.670,0,False,0
2,NaN,https://www.moneysmart.id/simak-info-penting-i...,2019-09-10,2019-09-10 00:00:00.599,2019-09-10 00:00:00.599,0,False,0
3,NaN,https://www.moneysmart.id/penghasilan-ump-mau-...,2019-09-10,2019-09-10 00:00:00.663,2019-09-10 00:00:00.663,0,False,0
4,NaN,https://www.moneysmart.id/7-resep-minuman-buka...,2019-09-10,2019-09-10 00:00:00.671,2019-09-10 00:00:00.671,0,False,0
5,NaN,https://www.moneysmart.id/reino-barack-dan-gay...,2019-09-10,2019-09-10 00:00:00.665,2019-09-10 00:00:00.665,0,False,0
6,NaN,https://www.moneysmart.id/8-daftar-headphone-t...,2019-09-10,2019-09-10 00:00:00.689,2019-09-10 00:00:00.689,0,False,0
7,NaN,https://www.moneysmart.id/kejarlah-kesuksesan-...,2019-09-10,2019-09-10 00:00:00.615,2019-09-10 00:00:00.615,0,False,0
8,NaN,https://www.moneysmart.id/category/smart-money...,2019-09-10,2019-09-10 00:00:00.490,2019-09-10 00:00:00.490,0,False,0
9,NaN,https://www.moneysmart.id/hak-dan-cara-kita-un...,2019-09-10,2019-09-10 00:00:00.607,2019-09-10 00:00:00.607,0,False,0


In [25]:
for c in df.dtypes: print(c)

object
object
object
object
object
int64
bool
int64


In [31]:
df = df.astype({
    "anonymous_id":"string_",
    "canonical_url":"string_",
    "day_str":"string_",
    "from_time":"datetime64",
   "to_time":"datetime64",
    "time_on_page_seconds":"float16",
    "is_test":"bool",
    "max_article_read_depth":"int16",
    
}, copy=False)


In [32]:
for c in df.dtypes: print(c)

object
object
object
datetime64[ns]
datetime64[ns]
float16
bool
int16


In [12]:
pyathena.__version__

'1.8.0'

In [34]:
# Run a test query to see that the function works

df = query_athena_via_s3(speed_test_query, athena_cursor, s3_client, print_debug_messages=True)
print("Num records = %i"% len(df))
df.head(10)

Running query
Query finished after 16.73
Now going to get the results from S3
Reading the CSV from S3 (not running query) took 272002.44
All finished using as_pandas in 18.89
Num records = 389406


,anonymous_id,canonical_url,day_str,from_time,to_time,time_on_page_seconds,is_test,max_article_read_depth
0,85da893c-245e-46db-9892-3e36d4a47fe0,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 22:34:44.142,2019-10-28 22:38:25.310,221,False,100
1,e083f2c1-7459-4bae-98e0-0de99464e816,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 22:56:40.997,2019-10-28 22:57:49.543,68,False,100
2,0581c41d-2310-45cc-9c95-c082e418de02,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 16:38:07.689,2019-10-28 16:39:21.032,73,False,100
3,0629cb02-8f5e-4a3d-bf6d-0820e116cb94,https://www.moneysmart.tw/articles/%E5%AD%98%E...,2019-10-28,2019-10-28 16:26:48.853,2019-10-28 16:27:25.412,36,False,100
4,8cc3e00a-6fe6-4c83-b6fd-bb06d0f328b6,https://www.moneysmart.tw/articles/%e9%89%85%e...,2019-10-28,2019-10-28 16:16:48.076,2019-10-28 16:16:48.076,0,False,0
5,66ee0aaf-15d7-49d8-89e1-80eeecc0dcc9,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 19:19:49.531,2019-10-28 19:20:11.895,22,False,100
6,50afd6f9-5cbc-49e9-bc2c-fba42a46cc59,https://www.moneysmart.tw/articles/netflix-lin...,2019-10-28,2019-10-28 19:36:28.890,2019-10-28 19:37:34.584,65,False,100
7,cc386a09-4cdd-4aca-b77e-139599922f24,https://www.moneysmart.tw/articles/%E5%9C%B0%E...,2019-10-28,2019-10-28 16:56:29.927,2019-10-28 16:56:29.951,0,False,100
8,b524ec2f-069b-4c82-a80f-d4008adc14f4,https://www.moneysmart.tw/articles/2019-%E6%84...,2019-10-28,2019-10-28 16:57:01.905,2019-10-28 16:57:01.905,0,False,0
9,3d14032a-6d30-4acd-bb26-944dcbd97a53,https://www.moneysmart.tw/articles/costco%E8%B...,2019-10-28,2019-10-28 22:11:53.677,2019-10-28 22:11:53.677,0,False,0


# Hacky AMP Pageviews Per Session Trend in AB Test


In [ ]:
athena = AthenaQuery()

# Get the Test URLs


# Generate a humongous query
# Pageviews per user who has visited an SEO test url.

# Get the results


